# Questionnaire Experiments - Research Replication Guide

This notebook demonstrates the questionnaire evaluation workflow from **"The Personality Trap"** research, continuing from the persona generation step.

## Overview

After generating AI personas, this workflow:

1. **Experiment Registration**: Create experiment groups and register individual runs
2. **Questionnaire Administration**: LLMs impersonate personas and answer personality questionnaires
3. **Response Logging**: Track all LLM requests/responses in the database
4. **Answer Processing**: Store and validate questionnaire answers
5. **Verification**: Inspect experiment results and data quality

## Prerequisites

✅ **Complete `personas_generation.ipynb` first** - you need generated personas to run experiments

✅ Database schema initialized with:
- `personas` table populated with generated personas
- `experiments_groups` table ready for experiment organization
- `experiments_list` table ready for individual experiment tracking
- `eval_questionnaires` table ready for storing LLM answers
- `experiment_request_metadata` table ready for API logs

✅ LLM API keys configured (same as persona generation)

---

## Step 1: Configure Database Access and Imports

Set up the environment for questionnaire experiments. This cell:

- Imports required libraries for experiment registration and execution
- Establishes database connection to your experimental schema
- Configures logging to track experiment progress
- Automatically loads configuration from `.yaml` in project root

**Note:** This assumes you've already run `personas_generation.ipynb` and have personas in your experimental schema.

**Database Architecture:**
The questionnaire experiment system uses two main tables:
1. `experiments_groups`: Defines batches of related experiments (what + how)
2. `experiments_list`: Individual experiment runs (who + results)

Each experiment in `experiments_list` links to its parent group via `experiments_group_id`.


In [1]:
import logging
from typing import List

import pandas as pd
from IPython.display import display
from sqlalchemy import text

# Import configuration and core modules
from personas_backend.utils.config import ConfigManager
from personas_backend.core.enums import ModelID
from personas_backend.db.db_handler import DatabaseHandler
from personas_backend.evaluate_questionnaire import (
    register_questionnaire_experiments,
    run_pending_experiments
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Load configuration (automatically finds .yaml in project root)
config = ConfigManager()

# Initialize database handler
db_handler = DatabaseHandler(config_manager=config)

# Get experimental schema from YAML config
EXPERIMENTAL_SCHEMA = config.schema_config.target_schema

print(f"✅ Configuration loaded successfully")
print(f"🧪 Experimental schema: {EXPERIMENTAL_SCHEMA}")
print(f"📊 Database connection established")


2025-10-09 18:22:58,644 - personas_backend.db.db_handler - INFO - Connected to the database!


✅ Configuration loaded successfully
🧪 Experimental schema: my_schema_v00
📊 Database connection established


## Step 2: Verify Generated Personas

Before registering experiments, confirm that personas exist in your experimental schema. This validation:

- Counts personas by model and population
- Identifies available `ref_personality_id` values for experiments
- Ensures the prerequisite data from `personas_generation.ipynb` is present

**If this cell shows no personas:** Run `personas_generation.ipynb` first to generate AI personas.

In [2]:
# Check for existing personas
personas_query = f"""
    SELECT
        COUNT(*) as total_personas,
        COUNT(DISTINCT model) as unique_models,
        COUNT(DISTINCT population) as unique_populations,
        COUNT(DISTINCT ref_personality_id) as unique_personalities
    FROM {EXPERIMENTAL_SCHEMA}.personas
"""

personas_summary = pd.read_sql_query(personas_query, db_handler.connection)
display(personas_summary)

if personas_summary['total_personas'].iloc[0] == 0:
    raise ValueError(
        f"No personas found in {EXPERIMENTAL_SCHEMA}.personas! "
        f"Run 'personas_generation.ipynb' first to generate personas."
    )

# Get detailed breakdown
personas_breakdown_query = f"""
    SELECT
        model,
        population,
        COUNT(*) as count
    FROM {EXPERIMENTAL_SCHEMA}.personas
    GROUP BY model, population
    ORDER BY model, population
"""

personas_breakdown = pd.read_sql_query(personas_breakdown_query, db_handler.connection)

print(f"\n✅ Found {personas_summary['total_personas'].iloc[0]} personas")
print(f"\n📊 Breakdown by model and population:")
display(personas_breakdown)

# Extract available personality IDs for experiment registration
available_personality_ids_query = f"""
    SELECT DISTINCT ref_personality_id
    FROM {EXPERIMENTAL_SCHEMA}.personas
    WHERE ref_personality_id IS NOT NULL
    ORDER BY ref_personality_id
"""

personality_ids_df = pd.read_sql_query(available_personality_ids_query, db_handler.connection)
available_personality_ids = personality_ids_df['ref_personality_id'].tolist()

print(f"\n🎯 Available personality IDs for experiments: {available_personality_ids}")

,total_personas,unique_models,unique_populations,unique_personalities
0,6,2,3,2



✅ Found 6 personas

📊 Breakdown by model and population:


,model,population,count
0,gpt35,generated_gpt35_spain826,2
1,gpt4o,borderline_maxN_gpt4o,2
2,gpt4o,borderline_maxP_gpt4o,2



🎯 Available personality IDs for experiments: [1, 2]


## Step 3: Auto-Discover and Register All Experiment Groups

**Automated Multi-Model Experiment Registration**

Instead of manually specifying models and populations, this step **automatically discovers** all generated personas and creates experiment groups for each unique model-population combination.

### How it works:

**Phase 1: Discovery**
- Query the `personas` table to find all unique (model, population) combinations
- Count personas in each combination
- Display a summary of what will be processed

**Phase 2: Batch Registration**
For each discovered model-population pair:
1. Create an experiment group in `experiments_groups` table
2. Register individual experiments in `experiments_list` (one per persona)
3. Track all group IDs and experiment IDs for later execution

### Architecture Pattern (Multi-Group)
```
experiments_groups
    ├── Group 1: gpt4o / generated_gpt4o_spain826 (baseline)
    │   ├── experiment_id: 1001, 1002, 1003...
    ├── Group 2: gpt35 / generated_gpt35_spain826 (baseline)  
    │   ├── experiment_id: 2001, 2002, 2003...
    ├── Group 3: gpt4o / borderline_maxN_gpt4o
    │   ├── experiment_id: 3001, 3002, 3003...
    └── Group 4: gpt4o / borderline_maxP_gpt4o
        └── experiment_id: 4001, 4002, 4003...
```

### Benefits:
- ✅ **No manual configuration** - automatically processes all generated personas
- ✅ **Handles baseline AND borderline** populations seamlessly
- ✅ **Supports multiple models** - GPT-3.5, GPT-4o, Claude, Llama, etc.
- ✅ **Fault tolerant** - continues if one registration fails

### Key Variables:
- `QUESTIONNAIRE_TYPE`: Which questionnaire to administer ('bigfive' or 'epqra')
- `ALL_EXPERIMENT_GROUPS`: List of all created group metadata
- `ALL_REGISTERED_EXPERIMENTS`: Complete list of experiment IDs across all groups

**Note**: Each group links to its specific model-population combination, ensuring experiments are run with the correct LLM.


In [3]:
# Step 3.1: Discover all generated persona populations automatically
print(f"🔍 Discovering all generated persona populations in {EXPERIMENTAL_SCHEMA}...")

# Query to get all unique model-population combinations
discovery_query = f"""
    SELECT 
        model,
        population,
        COUNT(*) as persona_count,
        COUNT(DISTINCT ref_personality_id) as unique_personalities
    FROM {EXPERIMENTAL_SCHEMA}.personas
    GROUP BY model, population
    ORDER BY model, population
"""

populations_df = pd.read_sql_query(discovery_query, db_handler.connection)

if populations_df.empty:
    raise ValueError(
        f"No personas found in {EXPERIMENTAL_SCHEMA}.personas! "
        f"Run 'personas_generation.ipynb' first to generate personas."
    )

print(f"\n✅ Found {len(populations_df)} model-population combinations:")
display(populations_df)

# Step 3.2: Configure experiment parameters
QUESTIONNAIRE_TYPE = "epqra"  # Options: 'bigfive', 'epqra'

print(f"\n🎯 Experiment Configuration:")
print(f"   • Questionnaire: {QUESTIONNAIRE_TYPE}")
print(f"   • Schema: {EXPERIMENTAL_SCHEMA}")
print(f"   • Total populations to process: {len(populations_df)}")

# Step 3.3: Register experiment groups for ALL model-population combinations
print(f"\n⏳ Registering experiment groups for all populations...")

# Store all experiment group IDs and experiment IDs
ALL_EXPERIMENT_GROUPS = []
ALL_REGISTERED_EXPERIMENTS = []

for idx, row in populations_df.iterrows():
    model = row['model']
    population = row['population']
    persona_count = row['persona_count']
    
    print(f"\n📋 Processing: {model} / {population} ({persona_count} personas)")
    
    try:
        group_id, experiment_ids = register_questionnaire_experiments(
            questionnaire=QUESTIONNAIRE_TYPE,
            model=model,
            populations=[population],
            experiment_description=f"Replication: {QUESTIONNAIRE_TYPE} for {model} / {population}",
            schema=EXPERIMENTAL_SCHEMA,
        )
        
        ALL_EXPERIMENT_GROUPS.append({
            'group_id': group_id,
            'model': model,
            'population': population,
            'experiment_count': len(experiment_ids)
        })
        ALL_REGISTERED_EXPERIMENTS.extend(experiment_ids)
        
        print(f"   ✅ Group {group_id}: {len(experiment_ids)} experiments registered")
        
    except Exception as e:
        print(f"   ❌ Failed to register {model}/{population}: {e}")
        # Continue with other populations even if one fails
        continue

# Convert to DataFrame for easy viewing
groups_summary_df = pd.DataFrame(ALL_EXPERIMENT_GROUPS)

print(f"\n🎉 Registration Summary:")
print(f"   • Total experiment groups created: {len(ALL_EXPERIMENT_GROUPS)}")
print(f"   • Total experiments registered: {len(ALL_REGISTERED_EXPERIMENTS)}")
print(f"\n📊 Experiment Groups:")
display(groups_summary_df)


🔍 Discovering all generated persona populations in my_schema_v00...

✅ Found 3 model-population combinations:


,model,population,persona_count,unique_personalities
0,gpt35,generated_gpt35_spain826,2,2
1,gpt4o,borderline_maxN_gpt4o,2,2
2,gpt4o,borderline_maxP_gpt4o,2,2



🎯 Experiment Configuration:
   • Questionnaire: epqra
   • Schema: my_schema_v00
   • Total populations to process: 3

⏳ Registering experiment groups for all populations...

📋 Processing: gpt35 / generated_gpt35_spain826 (2 personas)


2025-10-09 18:22:59,032 - personas_backend.evaluate_questionnaire.registration - INFO - Loaded 2 personas from schema=my_schema_v00 for model=gpt35 populations=['generated_gpt35_spain826']
2025-10-09 18:22:59,090 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,110 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment group with ID: 1 in schema: my_schema_v00


2025-10-09 18:22:59,181 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,223 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment id=1 in schema=my_schema_v00
2025-10-09 18:22:59,239 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment id=2 in schema=my_schema_v00
2025-10-09 18:22:59,250 - personas_backend.evaluate_questionnaire.registration - INFO - Loaded 2 personas from schema=my_schema_v00 for model=gpt4o populations=['borderline_maxN_gpt4o']
2025-10-09 18:22:59,308 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,321 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment group with ID: 2 in schema: my_schema_v00
2025-10-09 18:22:59,379 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,396 - p

   ✅ Group 1: 2 experiments registered

📋 Processing: gpt4o / borderline_maxN_gpt4o (2 personas)
   ✅ Group 2: 2 experiments registered

📋 Processing: gpt4o / borderline_maxP_gpt4o (2 personas)


2025-10-09 18:22:59,513 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,525 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment group with ID: 3 in schema: my_schema_v00
2025-10-09 18:22:59,629 - personas_backend.evaluate_questionnaire.registration - INFO - Connected to the database!
2025-10-09 18:22:59,670 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment id=5 in schema=my_schema_v00
2025-10-09 18:22:59,712 - personas_backend.evaluate_questionnaire.registration - INFO - Registered new experiment id=6 in schema=my_schema_v00


   ✅ Group 3: 2 experiments registered

🎉 Registration Summary:
   • Total experiment groups created: 3
   • Total experiments registered: 6

📊 Experiment Groups:


,group_id,model,population,experiment_count
0,1,gpt35,generated_gpt35_spain826,2
1,2,gpt4o,borderline_maxN_gpt4o,2
2,3,gpt4o,borderline_maxP_gpt4o,2


## Step 3.5: Understanding Repetitions and Experiment Architecture

**How repetitions work:**

### Persona-Level Repetitions (Automatic)
Each persona in the database has a `repetitions` field (default: 1) that indicates how many times to repeat experiments for that persona. This is used for statistical reliability testing.

**Example:**
- Persona #1 has `repetitions = 3`
- Persona #2 has `repetitions = 1`
- Registration creates: 4 experiments total (3 for persona #1, 1 for persona #2)

### Registration Flow
```python
register_questionnaire_experiments(
    questionnaire="bigfive",
    model="gpt4o",
    populations=["generated_gpt4o_spain826"]
)
```

**What happens internally:**
1. **Fetch personas**: Query `personas` table filtered by model + populations
2. **Create experiment group**: One record in `experiments_groups` with:
   - Description, system_role, base_prompt (questionnaire JSON)
   - LLM parameters (temperature, top_p)
   - Metadata (created timestamp, git SHA)
3. **Register experiments**: For each persona:
   - Read `persona.repetitions` (how many times to repeat)
   - Create N experiments in `experiments_list` (one per repetition)
   - Each links to group via `experiments_group_id`
4. **Return**: `(group_id, list_of_experiment_ids)`

### Database Relationships
```
experiments_groups (1 record)
    ├── experiments_list (N records, N = sum of all persona.repetitions)
    │   ├── experiment_id: 1001 (persona=42, repetition=1)
    │   ├── experiment_id: 1002 (persona=42, repetition=2)  
    │   ├── experiment_id: 1003 (persona=42, repetition=3)
    │   └── experiment_id: 1004 (persona=99, repetition=1)
```

**For this research replication:**
- All baseline personas have `repetitions = 1` (one experiment per persona)
- Borderline personas also have `repetitions = 1`
- Advanced users can set `max_repetitions` parameter to cap experiments (useful for testing)


## Step 4: Verify All Experiment Group Registrations

Inspect the registered experiment groups before execution. This validation shows:

- **All experiment groups** created across different models and populations
- Group metadata (description, parameters, prompts)
- Individual experiment records per group
- Current status (should all be `succeeded=NULL` for pending experiments)

This comprehensive view lets you:
- Confirm all model-population combinations were registered
- Verify experiment counts match expectations
- Review the architecture before running expensive LLM API calls

**What to check:**
- Each model-population pair has its own experiment group
- Experiment count per group = number of personas in that population
- Baseline AND borderline populations are represented (if generated)


In [4]:
# Optional: Explore the relationship between experiment groups and experiments
SHOW_ARCHITECTURE = True  # Set to True to see how groups and experiments relate

if SHOW_ARCHITECTURE:
    # Query to show the structure
    architecture_query = f"""
        SELECT 
            eg.experiments_group_id,
            LEFT(eg.description, 50) || '...' as description_preview,
            eg.temperature,
            eg.top_p,
            COUNT(el.experiment_id) as total_experiments,
            COUNT(DISTINCT el.population) as unique_populations,
            COUNT(DISTINCT el.personality_id) as unique_personas
        FROM {EXPERIMENTAL_SCHEMA}.experiments_groups eg
        LEFT JOIN {EXPERIMENTAL_SCHEMA}.experiments_list el 
            ON eg.experiments_group_id = el.experiments_group_id
        GROUP BY eg.experiments_group_id, eg.description, eg.temperature, eg.top_p
        ORDER BY eg.experiments_group_id DESC
        LIMIT 5
    """
    
    architecture_df = pd.read_sql_query(architecture_query, db_handler.connection)
    
    print(f"📊 Experiment Group → Experiments Relationship:")
    print(f"   (Showing most recent 5 groups)\n")
    display(architecture_df)
    
    print(f"\n💡 Interpretation:")
    print(f"   • Each row = 1 experiment group")
    print(f"   • total_experiments = number of individual experiment runs")
    print(f"   • unique_personas = distinct personalities being tested")
    print(f"   • Relationship: 1 group has N experiments (one per persona)")
else:
    print(f"ℹ️  Architecture exploration skipped.")
    print(f"   Set SHOW_ARCHITECTURE=True to see experiment group structure.")


📊 Experiment Group → Experiments Relationship:
   (Showing most recent 5 groups)



,experiments_group_id,description_preview,temperature,top_p,total_experiments,unique_populations,unique_personas
0,3,Replication: epqra for gpt4o / borderline_maxP...,1.0,1.0,2,1,2
1,2,Replication: epqra for gpt4o / borderline_maxN...,1.0,1.0,2,1,2
2,1,Replication: epqra for gpt35 / generated_gpt35...,1.0,1.0,2,1,2



💡 Interpretation:
   • Each row = 1 experiment group
   • total_experiments = number of individual experiment runs
   • unique_personas = distinct personalities being tested
   • Relationship: 1 group has N experiments (one per persona)


In [5]:
# Inspect ALL experiment groups in detail
all_group_ids_str = ', '.join(str(g['group_id']) for g in ALL_EXPERIMENT_GROUPS)

# Query all experiment groups
groups_query = f"""
    SELECT *
    FROM {EXPERIMENTAL_SCHEMA}.experiments_groups
    WHERE experiments_group_id IN ({all_group_ids_str})
    ORDER BY experiments_group_id
"""

all_groups_df = pd.read_sql_query(groups_query, db_handler.connection)

print(f"📋 All Registered Experiment Groups ({len(all_groups_df)}):")
print(f"\nShowing key fields (transpose for readability):\n")

# Display each group's key info
for idx, row in all_groups_df.iterrows():
    print(f"{'='*80}")
    print(f"Group {row['experiments_group_id']}: {row.get('description', 'N/A')[:60]}...")
    print(f"   • Temperature: {row.get('temperature', 'N/A')}")
    print(f"   • Top-p: {row.get('top_p', 'N/A')}")
    print(f"   • Created: {row.get('created', 'N/A')}")

# Query all registered experiments across all groups
experiments_query = f"""
    SELECT
        experiment_id,
        experiments_group_id,
        questionnaire,
        model,
        population,
        personality_id,
        repeated,
        succeeded,
        created
    FROM {EXPERIMENTAL_SCHEMA}.experiments_list
    WHERE experiments_group_id IN ({all_group_ids_str})
    ORDER BY experiments_group_id, experiment_id
"""

all_experiments_df = pd.read_sql_query(experiments_query, db_handler.connection)

print(f"\n{'='*80}")
print(f"\n📊 All Registered Experiments (showing first 20):")
display(all_experiments_df.head(20))

# Summary by group
print(f"\n📈 Summary by Experiment Group:")
group_exp_summary = all_experiments_df.groupby(['experiments_group_id', 'model', 'population']).agg({
    'experiment_id': 'count',
    'succeeded': lambda x: (x == True).sum()
}).rename(columns={'experiment_id': 'total_experiments', 'succeeded': 'succeeded'})
group_exp_summary['pending'] = all_experiments_df.groupby(['experiments_group_id', 'model', 'population'])['succeeded'].apply(lambda x: x.isna().sum())

display(group_exp_summary)

print(f"\n✅ Overall Registration Summary:")
print(f"   • Total experiment groups: {len(all_groups_df)}")
print(f"   • Total experiments registered: {len(all_experiments_df)}")
print(f"   • Pending (not yet run): {all_experiments_df['succeeded'].isna().sum()}")
print(f"   • Succeeded: {(all_experiments_df['succeeded'] == True).sum()}")
print(f"   • Failed: {(all_experiments_df['succeeded'] == False).sum()}")


📋 All Registered Experiment Groups (3):

Showing key fields (transpose for readability):

Group 1: Replication: epqra for gpt35 / generated_gpt35_spain826...
   • Temperature: 1.0
   • Top-p: 1.0
   • Created: 2025-10-09T17:22:59.093105
Group 2: Replication: epqra for gpt4o / borderline_maxN_gpt4o...
   • Temperature: 1.0
   • Top-p: 1.0
   • Created: 2025-10-09T17:22:59.310968
Group 3: Replication: epqra for gpt4o / borderline_maxP_gpt4o...
   • Temperature: 1.0
   • Top-p: 1.0
   • Created: 2025-10-09T17:22:59.515859


📊 All Registered Experiments (showing first 20):


,experiment_id,experiments_group_id,questionnaire,model,population,personality_id,repeated,succeeded,created
0,1,1,epqra,gpt-3.5-turbo-0125,generated_gpt35_spain826,1,1,None,2025-10-09T17:22:59.207402
1,2,1,epqra,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,1,None,2025-10-09T17:22:59.231279
2,3,2,epqra,gpt-4o-2024-11-20,borderline_maxN_gpt4o,2,1,None,2025-10-09T17:22:59.386591
3,4,2,epqra,gpt-4o-2024-11-20,borderline_maxN_gpt4o,1,1,None,2025-10-09T17:22:59.406758
4,5,3,epqra,gpt-4o-2024-11-20,borderline_maxP_gpt4o,2,1,None,2025-10-09T17:22:59.639785
5,6,3,epqra,gpt-4o-2024-11-20,borderline_maxP_gpt4o,1,1,None,2025-10-09T17:22:59.689474



📈 Summary by Experiment Group:


,,,total_experiments,succeeded,pending
experiments_group_id,model,population,,,
1,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,0,2
2,gpt-4o-2024-11-20,borderline_maxN_gpt4o,2,0,2
3,gpt-4o-2024-11-20,borderline_maxP_gpt4o,2,0,2



✅ Overall Registration Summary:
   • Total experiment groups: 3
   • Total experiments registered: 6
   • Pending (not yet run): 6
   • Succeeded: 0
   • Failed: 0


## Step 5: Execute ALL Pending Experiments Across All Groups

**Multi-Model Batch Execution** - This runs experiments for ALL model-population combinations simultaneously.

**What happens during execution:**

1. **For each experiment group:**
   - System processes experiments using the group's assigned model (GPT-4o, GPT-3.5, Claude, etc.)
   - Each experiment impersonates its specific persona
   - Questionnaire questions are sent sequentially to the appropriate LLM
   - Answers are parsed and validated (1-5 scale)

2. **Concurrent processing:**
   - Multiple experiments run in parallel (controlled by `max_workers`)
   - Different models can be queried simultaneously
   - Progress logged in real-time for all groups

3. **Data logging (automatic for all groups):**
   - `experiment_request_metadata`: Full LLM API request/response JSON (per model)
   - `eval_questionnaires`: Parsed questionnaire answers
   - `experiments_list`: Status updates with `succeeded` flags

4. **Error handling:**
   - API failures logged per experiment
   - One failed experiment doesn't halt the entire batch
   - Can retry failed experiments later

**Performance tuning:**
- `max_workers=3`: Concurrent experiments (adjust for API rate limits)
- `batch_size=None`: Process ALL pending experiments
- Consider reducing `max_workers` if hitting rate limits across multiple API keys

**Cost Warning:** 
- This executes experiments for **ALL models and populations**
- Total API calls = (# of experiments) × (~50 questions) × (# of groups)
- Verify API quotas for ALL models (OpenAI, Bedrock, etc.) before proceeding
- Each model incurs its own API costs

**Execution time estimate:**
- GPT-4o: ~30-60 seconds per experiment
- GPT-3.5: ~20-40 seconds per experiment  
- Claude/Llama: Variable based on Bedrock configuration
- Total time = (total experiments) / max_workers × avg_time_per_experiment


In [6]:
# Configuration for experiment execution
MAX_WORKERS = 3  # Concurrent experiment workers (adjust based on API rate limits)
BATCH_SIZE = None  # None = process all; set to integer to limit batch size

# Get all experiment group IDs
all_group_ids = [group['group_id'] for group in ALL_EXPERIMENT_GROUPS]

print(f"🎯 Experiment Execution Configuration:")
print(f"   • Number of experiment groups: {len(all_group_ids)}")
print(f"   • Total pending experiments: {len(ALL_REGISTERED_EXPERIMENTS)}")
print(f"   • Max workers: {MAX_WORKERS}")
print(f"   • Batch size: {'All' if BATCH_SIZE is None else BATCH_SIZE}")

print(f"\n📋 Experiment groups to execute:")
for group in ALL_EXPERIMENT_GROUPS:
    print(f"   • Group {group['group_id']}: {group['model']} / {group['population']} ({group['experiment_count']} experiments)")

# Confirm execution
print(f"\n⚠️  Warning: This will make LLM API calls and may incur costs.")
print(f"   • Total experiments: {len(ALL_REGISTERED_EXPERIMENTS)}")
print(f"   • Estimated API calls: {len(ALL_REGISTERED_EXPERIMENTS)} experiments × ~50 questions = ~{len(ALL_REGISTERED_EXPERIMENTS) * 50} calls")
print(f"   • Check your API quotas and rate limits before proceeding.")

CONFIRM_EXECUTION = True  # Set to True after reviewing the warning

if not CONFIRM_EXECUTION:
    print(f"\n⏸️  Execution paused. Set CONFIRM_EXECUTION=True to proceed.")
else:
    print(f"\n⏳ Starting experiment execution for ALL groups...")
    print(f"   Monitor logs for progress and errors.\n")
    
    try:
        # Execute all experiment groups in one batch
        run_pending_experiments(
            experiments_group_ids=all_group_ids,  # Pass ALL group IDs
            batch_size=BATCH_SIZE,
            max_workers=MAX_WORKERS,
            schema=EXPERIMENTAL_SCHEMA,
        )
        
        print(f"\n✅ All experiment groups execution completed!")
        print(f"   • Processed {len(all_group_ids)} experiment groups")
        print(f"   • Total experiments: {len(ALL_REGISTERED_EXPERIMENTS)}")
        print(f"   Check the verification cells below for results.")
        
    except Exception as e:
        print(f"\n❌ Experiment execution failed: {e}")
        print(f"   Common issues:")
        print(f"   • API key missing or invalid")
        print(f"   • API rate limit exceeded")
        print(f"   • Network connectivity issues")
        print(f"   • LLM response parsing errors")
        print(f"\n   Check logs for detailed error messages.")
        print(f"   Failed experiments can be retried by re-running this cell.")
        raise


🎯 Experiment Execution Configuration:
   • Number of experiment groups: 3
   • Total pending experiments: 6
   • Max workers: 3
   • Batch size: All

📋 Experiment groups to execute:
   • Group 1: gpt35 / generated_gpt35_spain826 (2 experiments)
   • Group 2: gpt4o / borderline_maxN_gpt4o (2 experiments)
   • Group 3: gpt4o / borderline_maxP_gpt4o (2 experiments)

⚠️  Warning: This will make LLM API calls and may incur costs.
   • Total experiments: 6
   • Estimated API calls: 6 experiments × ~50 questions = ~300 calls
   • Check your API quotas and rate limits before proceeding.

⏳ Starting experiment execution for ALL groups...
   Monitor logs for progress and errors.



2025-10-09 18:23:00,440 - evaluate_questionnaire_runner - INFO - Logger evaluate_questionnaire_runner is set up and ready to log.
2025-10-09 18:23:00,440 - evaluate_questionnaire_runner - INFO - Logger evaluate_questionnaire_runner is set up and ready to log.


2025-10-09 18:23:00,518 - evaluate_questionnaire_runner - INFO - Connected to the database!
2025-10-09 18:23:00,518 - evaluate_questionnaire_runner - INFO - Connected to the database!
2025-10-09 18:23:00,641 - evaluate_questionnaire_runner - INFO - Processing experiments for experiments_group_id 3
2025-10-09 18:23:00,641 - evaluate_questionnaire_runner - INFO - Processing experiments for experiments_group_id 3
2025-10-09 18:23:00,646 - evaluate_questionnaire_runner - INFO - Found 2 experiments to process
2025-10-09 18:23:00,646 - evaluate_questionnaire_runner - INFO - Found 2 experiments to process
2025-10-09 18:23:00,650 - evaluate_questionnaire_runner - INFO - Model: gpt4o, Population: borderline_maxP_gpt4o, Questionnaire: epqra
2025-10-09 18:23:00,650 - evaluate_questionnaire_runner - INFO - Model: gpt4o, Population: borderline_maxP_gpt4o, Questionnaire: epqra
2025-10-09 18:23:00,918 - evaluate_questionnaire_runner - INFO - Submitting experiment 6 for personality 1
2025-10-09 18:23:


✅ All experiment groups execution completed!
   • Processed 3 experiment groups
   • Total experiments: 6
   Check the verification cells below for results.


## Step 6: Verify All Experiment Completions Across Groups

After execution, check results for **all experiment groups** across all models and populations. This helps identify:

- **Overall success rate** across all experiments
- **Per-group success rates** (model-population breakdown)
- Failed experiments by model (API issues, parsing failures, etc.)
- Pending experiments that need retry

**What to look for:**

**Overall Metrics:**
- Total experiments executed across all groups
- Success percentage (should be >95% ideally)
- Distribution of failures by model

**Per-Group Breakdown:**
- Success rate for each model-population combination
- Identify which models/populations had issues
- Compare baseline vs borderline completion rates

**Interpretation:**
- `succeeded=TRUE`: Experiment completed, answers stored
- `succeeded=FALSE`: Experiment failed, check `llm_explanation` for reason
- `succeeded=NULL`: Not yet executed (retry needed)

**Common patterns:**
- GPT models usually have higher success rates
- Borderline personas may have slightly higher failure rates (extreme responses)
- Rate limit errors appear as clusters of failures in specific time windows


## Step 6: Verify Experiment Completion

Confirm that the experiments completed successfully and inspect any reported errors. This cell shows:

- Success rate summary
- Status of each experiment
- LLM explanations for completed experiments
- Failed experiments (if any) for debugging


In [7]:
# Query experiment status for ALL experiment groups
all_group_ids_str = ', '.join(str(g['group_id']) for g in ALL_EXPERIMENT_GROUPS)

status_query = f"""
    SELECT
        el.experiments_group_id,
        el.experiment_id,
        el.questionnaire,
        el.model,
        el.population,
        el.personality_id,
        el.repeated,
        el.succeeded,
        el.llm_explanation,
        el.created
    FROM {EXPERIMENTAL_SCHEMA}.experiments_list el
    WHERE el.experiments_group_id IN ({all_group_ids_str})
    ORDER BY el.experiments_group_id, el.experiment_id
"""

experiment_status = pd.read_sql_query(status_query, db_handler.connection)

# Overall summary statistics
total_experiments = len(experiment_status)
succeeded_count = (experiment_status['succeeded'] == True).sum()
failed_count = (experiment_status['succeeded'] == False).sum()
pending_count = experiment_status['succeeded'].isna().sum()

print(f"📊 Overall Experiment Status Summary:")
print(f"   • Total experiments: {total_experiments}")
print(f"   • ✅ Succeeded: {succeeded_count} ({succeeded_count/total_experiments*100:.1f}%)")
print(f"   • ❌ Failed: {failed_count} ({failed_count/total_experiments*100:.1f}%)")
print(f"   • ⏳ Pending: {pending_count} ({pending_count/total_experiments*100:.1f}%)")

# Per-group summary
print(f"\n📋 Status by Experiment Group:")
group_summary = experiment_status.groupby(['experiments_group_id', 'model', 'population']).agg({
    'experiment_id': 'count',
    'succeeded': lambda x: (x == True).sum()
}).rename(columns={'experiment_id': 'total', 'succeeded': 'succeeded'})
group_summary['failed'] = experiment_status.groupby(['experiments_group_id', 'model', 'population'])['succeeded'].apply(lambda x: (x == False).sum())
group_summary['pending'] = experiment_status.groupby(['experiments_group_id', 'model', 'population'])['succeeded'].apply(lambda x: x.isna().sum())
group_summary['success_rate'] = (group_summary['succeeded'] / group_summary['total'] * 100).round(1)

display(group_summary)

# Display sample results from all groups
print(f"\n📝 Sample Experiment Status (first 20 across all groups):")
display(experiment_status[['experiments_group_id', 'experiment_id', 'model', 'population', 'personality_id', 'succeeded']].head(20))

# Show failed experiments if any
if failed_count > 0:
    print(f"\n⚠️  Failed Experiments:")
    failed_experiments = experiment_status[experiment_status['succeeded'] == False]
    display(failed_experiments[['experiments_group_id', 'experiment_id', 'model', 'population', 'personality_id', 'llm_explanation']])
    
    print(f"\nℹ️  You can retry failed experiments by re-running the execution cell.")

# Check if all experiments completed
if pending_count > 0:
    print(f"\n⚠️  {pending_count} experiments still pending!")
    print(f"   Re-run the execution cell (Step 5) to process them.")


📊 Overall Experiment Status Summary:
   • Total experiments: 6
   • ✅ Succeeded: 6 (100.0%)
   • ❌ Failed: 0 (0.0%)
   • ⏳ Pending: 0 (0.0%)

📋 Status by Experiment Group:


,,,total,succeeded,failed,pending,success_rate
experiments_group_id,model,population,,,,,
1,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,2,0,0,100.0
2,gpt-4o-2024-11-20,borderline_maxN_gpt4o,2,2,0,0,100.0
3,gpt-4o-2024-11-20,borderline_maxP_gpt4o,2,2,0,0,100.0



📝 Sample Experiment Status (first 20 across all groups):


,experiments_group_id,experiment_id,model,population,personality_id,succeeded
0,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,1,True
1,1,2,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,True
2,2,3,gpt-4o-2024-11-20,borderline_maxN_gpt4o,2,True
3,2,4,gpt-4o-2024-11-20,borderline_maxN_gpt4o,1,True
4,3,5,gpt-4o-2024-11-20,borderline_maxP_gpt4o,2,True
5,3,6,gpt-4o-2024-11-20,borderline_maxP_gpt4o,1,True


## Step 7: Review Questionnaire Answers Across All Models

Inspect questionnaire responses collected from **all LLMs across all experiment groups**. This comprehensive view shows:

- **Answer distribution** by model and population
- **Answer completeness** (questions per experiment)
- **Cross-model comparisons** of response patterns
- **Baseline vs borderline** response differences

**Database aggregation:**
- Joins `eval_questionnaires` + `experiments_list` for full context
- Groups by model, population, experiment group
- Enables cross-model analysis

**Key analyses:**

1. **Model-Population Breakdown**
   - How many answers per model-population combination
   - Average completion rate by model
   - Identify missing data

2. **Answer Distribution (1-5 scale)**
   - Overall response patterns across all models
   - Model-specific biases (e.g., GPT-4o vs GPT-3.5)
   - Baseline vs borderline response differences

3. **Completeness Validation**
   - Expected questions: 50 (Big Five) or 24 (EPQ-R-A)
   - Detect incomplete experiments
   - Flag parsing errors

**Use for analysis:**
- Calculate Big Five personality scores per model
- Compare trait distributions across models
- Analyze how different LLMs interpret the same persona
- Study borderline condition effects on responses


In [8]:
# Get ALL experiment IDs across all groups
all_experiment_ids_str = ', '.join(str(eid) for eid in ALL_REGISTERED_EXPERIMENTS)

# Query questionnaire answers for ALL experiments
answers_query = f"""
    SELECT
        eq.experiment_id,
        eq.question_number,
        eq.answer,
        el.model,
        el.population,
        el.personality_id,
        el.experiments_group_id
    FROM {EXPERIMENTAL_SCHEMA}.eval_questionnaires eq
    JOIN {EXPERIMENTAL_SCHEMA}.experiments_list el
        ON eq.experiment_id = el.experiment_id
    WHERE eq.experiment_id IN ({all_experiment_ids_str})
    ORDER BY el.experiments_group_id, eq.experiment_id, eq.question_number
"""

answers_df = pd.read_sql_query(answers_query, db_handler.connection)

if answers_df.empty:
    print(f"⚠️  No answers found!")
    print(f"   Possible reasons:")
    print(f"   • Experiments haven't been executed yet")
    print(f"   • All experiments failed (check status in previous cell)")
    print(f"   • Database query error")
else:
    total_answers = len(answers_df)
    unique_experiments = answers_df['experiment_id'].nunique()
    unique_groups = answers_df['experiments_group_id'].nunique()
    avg_answers_per_experiment = total_answers / unique_experiments if unique_experiments > 0 else 0
    
    print(f"✅ Found {total_answers} questionnaire answers")
    print(f"   • From {unique_experiments} experiments")
    print(f"   • Across {unique_groups} experiment groups")
    print(f"   • Average answers per experiment: {avg_answers_per_experiment:.1f}")
    
    # Answer distribution by model and population
    print(f"\n📊 Answers by Model and Population:")
    model_pop_summary = answers_df.groupby(['model', 'population']).agg({
        'answer': 'count',
        'experiment_id': 'nunique'
    }).rename(columns={'answer': 'total_answers', 'experiment_id': 'experiments'})
    model_pop_summary['avg_answers'] = (model_pop_summary['total_answers'] / model_pop_summary['experiments']).round(1)
    display(model_pop_summary)
    
    # Overall answer distribution analysis
    print(f"\n📊 Overall Answer Distribution (1-5 scale):")
    answer_dist = answers_df['answer'].value_counts().sort_index()
    for value, count in answer_dist.items():
        percentage = count / total_answers * 100
        print(f"   • {value}: {count:5d} ({percentage:5.1f}%)")
    
    # Check completeness dynamically based on most common answer count
    answers_per_exp = answers_df.groupby('experiment_id').size()
    expected_questions = answers_per_exp.mode()[0]  # Most common count
    complete_experiments = (answers_per_exp == expected_questions).sum()
    
    print(f"\n📋 Answer Completeness:")
    print(f"   • Expected questions per experiment: {expected_questions}")
    print(f"   • Complete experiments: {complete_experiments}")
    print(f"   • Incomplete experiments: {unique_experiments - complete_experiments}")
    
    if (unique_experiments - complete_experiments) > 0:
        print(f"\n⚠️  Some experiments have unexpected answer counts:")
        incomplete = answers_per_exp[answers_per_exp != expected_questions]
        incomplete_df = pd.DataFrame({
            'experiment_id': incomplete.index, 
            'answer_count': incomplete.values
        })
        # Add model/population info for context
        incomplete_with_info = incomplete_df.merge(
            answers_df[['experiment_id', 'model', 'population']].drop_duplicates(),
            on='experiment_id'
        )
        display(incomplete_with_info)
    
    # Display sample answers from multiple groups
    print(f"\n📝 Sample Answers (first 20 rows across all groups):")
    display(answers_df[['experiments_group_id', 'experiment_id', 'model', 'population', 'question_number', 'answer']].head(20))


✅ Found 144 questionnaire answers
   • From 6 experiments
   • Across 3 experiment groups
   • Average answers per experiment: 24.0

📊 Answers by Model and Population:


total_answers  experiments  \
model              population                                             
gpt-3.5-turbo-0125 generated_gpt35_spain826             48            2   
gpt-4o-2024-11-20  borderline_maxN_gpt4o                48            2   
                   borderline_maxP_gpt4o                48            2   

                                             avg_answers  
model              population                             
gpt-3.5-turbo-0125 generated_gpt35_spain826         24.0  
gpt-4o-2024-11-20  borderline_maxN_gpt4o            24.0  
                   borderline_maxP_gpt4o            24.0


📊 Overall Answer Distribution (1-5 scale):
   • 0:    71 ( 49.3%)
   • 1:    73 ( 50.7%)

📋 Answer Completeness:
   • Expected questions per experiment: 24
   • Complete experiments: 6
   • Incomplete experiments: 0

📝 Sample Answers (first 20 rows across all groups):


,experiments_group_id,experiment_id,model,population,question_number,answer
0,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,1,1
1,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,0
2,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,3,1
3,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,4,0
4,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,5,0
5,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,6,0
6,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,7,0
7,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,8,1
8,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,9,1
9,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,10,0


## Step 8: Answer Summary Statistics Across All Groups

Aggregate answer statistics to understand the **complete dataset** across all models and populations:

**Per-Experiment Summary:**
- Answer counts for each experiment
- Grouped by model and population
- Identify experiments with missing answers

**Overall Statistics:**
- Total answers collected across all groups
- Average answers per experiment
- Min/max answer counts (quality check)

**Model-Population Breakdown:**
- Experiments completed per model-population pair
- Total answers per combination
- Average completeness rate

**Quality Metrics:**
- All experiments should have consistent answer counts
- Big Five: 50 answers per experiment
- EPQ-R-A: 24 answers per experiment
- Outliers indicate parsing or API issues

This summary helps:
- Verify data completeness before analysis
- Identify systematic issues with specific models
- Confirm baseline and borderline populations are represented
- Plan next steps for statistical analysis


In [9]:
# Aggregate answers by experiment across ALL groups
summary_query = f"""
    SELECT
        eq.experiment_id,
        el.experiments_group_id,
        el.model,
        el.population,
        el.personality_id,
        COUNT(*) AS answers_recorded
    FROM {EXPERIMENTAL_SCHEMA}.eval_questionnaires eq
    JOIN {EXPERIMENTAL_SCHEMA}.experiments_list el
        ON eq.experiment_id = el.experiment_id
    WHERE eq.experiment_id IN ({all_experiment_ids_str})
    GROUP BY eq.experiment_id, el.experiments_group_id, el.model, el.population, el.personality_id
    ORDER BY el.experiments_group_id, eq.experiment_id
"""

summary_df = pd.read_sql_query(summary_query, db_handler.connection)

print(f"📊 Answer Summary per Experiment (showing first 20):")
display(summary_df.head(20))

if not summary_df.empty:
    print(f"\n✅ Overall Statistics:")
    print(f"   • Total experiments with answers: {len(summary_df)}")
    print(f"   • Total answers collected: {summary_df['answers_recorded'].sum()}")
    print(f"   • Average answers per experiment: {summary_df['answers_recorded'].mean():.1f}")
    print(f"   • Min answers in any experiment: {summary_df['answers_recorded'].min()}")
    print(f"   • Max answers in any experiment: {summary_df['answers_recorded'].max()}")
    
    # Summary by model-population
    print(f"\n📊 Summary by Model and Population:")
    model_pop_stats = summary_df.groupby(['model', 'population']).agg({
        'experiment_id': 'count',
        'answers_recorded': ['sum', 'mean']
    })
    model_pop_stats.columns = ['experiments', 'total_answers', 'avg_answers']
    model_pop_stats['avg_answers'] = model_pop_stats['avg_answers'].round(1)
    display(model_pop_stats)


📊 Answer Summary per Experiment (showing first 20):


,experiment_id,experiments_group_id,model,population,personality_id,answers_recorded
0,1,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,1,24
1,2,1,gpt-3.5-turbo-0125,generated_gpt35_spain826,2,24
2,3,2,gpt-4o-2024-11-20,borderline_maxN_gpt4o,2,24
3,4,2,gpt-4o-2024-11-20,borderline_maxN_gpt4o,1,24
4,5,3,gpt-4o-2024-11-20,borderline_maxP_gpt4o,2,24
5,6,3,gpt-4o-2024-11-20,borderline_maxP_gpt4o,1,24



✅ Overall Statistics:
   • Total experiments with answers: 6
   • Total answers collected: 144
   • Average answers per experiment: 24.0
   • Min answers in any experiment: 24
   • Max answers in any experiment: 24

📊 Summary by Model and Population:


experiments  total_answers  \
model              population                                             
gpt-3.5-turbo-0125 generated_gpt35_spain826            2             48   
gpt-4o-2024-11-20  borderline_maxN_gpt4o               2             48   
                   borderline_maxP_gpt4o               2             48   

                                             avg_answers  
model              population                             
gpt-3.5-turbo-0125 generated_gpt35_spain826         24.0  
gpt-4o-2024-11-20  borderline_maxN_gpt4o            24.0  
                   borderline_maxP_gpt4o            24.0

## Step 9: Inspect LLM Request/Response Metadata (Optional)

For debugging or detailed analysis across **all models**, inspect raw LLM API calls stored in `experiment_request_metadata`.

**Multi-Model Metadata:**
- Request/response JSON for each model (GPT, Claude, Llama)
- Token usage by model (important for cost tracking)
- API call patterns and timing
- Model-specific metadata fields

**Use cases:**

1. **Cost Analysis**
   - Track token usage per model
   - Compare GPT-4o vs GPT-3.5 efficiency
   - Estimate future experiment costs

2. **Debugging**
   - Inspect failed experiment request/response pairs
   - Compare successful vs failed API calls
   - Identify model-specific parsing issues

3. **Performance Analysis**
   - Response times by model
   - Token efficiency (tokens per answer)
   - API throughput patterns

4. **Research Reproducibility**
   - Full API request history
   - Exact prompts sent to each model
   - Complete LLM responses for verification

**Token usage insights:**
- GPT-4o: Higher cost per token, better quality
- GPT-3.5: Lower cost, faster responses
- Claude/Llama: Variable based on Bedrock configuration

**Note:** Enable `INSPECT_METADATA=True` to view detailed API metadata and token statistics by model.


In [10]:
# Query request/response metadata (summary only, not full JSON)
INSPECT_METADATA = False  # Set to True to enable this optional inspection

if INSPECT_METADATA:
    metadata_query = f"""
        SELECT
            erm.id,
            erm.experiment_id,
            el.experiments_group_id,
            el.model,
            el.population,
            erm.created,
            jsonb_array_length(erm.request_json->'messages') as message_count,
            erm.response_json->'usage'->>'total_tokens' as total_tokens,
            erm.response_json->'usage'->>'prompt_tokens' as prompt_tokens,
            erm.response_json->'usage'->>'completion_tokens' as completion_tokens
        FROM {EXPERIMENTAL_SCHEMA}.experiment_request_metadata erm
        JOIN {EXPERIMENTAL_SCHEMA}.experiments_list el
            ON erm.experiment_id = el.experiment_id
        WHERE erm.experiment_id IN ({all_experiment_ids_str})
        ORDER BY erm.created DESC
        LIMIT 20
    """
    
    metadata_df = pd.read_sql_query(metadata_query, db_handler.connection)
    
    if not metadata_df.empty:
        print(f"📡 LLM API Request/Response Metadata (sample - 20 most recent):")
        display(metadata_df)
        
        # Token usage summary by model
        if 'total_tokens' in metadata_df.columns:
            print(f"\n📊 Token Usage Summary:")
            token_summary = metadata_df.groupby('model').agg({
                'total_tokens': lambda x: x.astype(float).sum(),
                'prompt_tokens': lambda x: x.astype(float).sum(),
                'completion_tokens': lambda x: x.astype(float).sum(),
                'id': 'count'
            }).rename(columns={'id': 'api_calls'})
            
            for col in ['total_tokens', 'prompt_tokens', 'completion_tokens']:
                token_summary[col] = token_summary[col].astype(int)
            
            display(token_summary)
            
            total_tokens_all = metadata_df['total_tokens'].astype(float).sum()
            print(f"\n   💰 Total tokens used (sampled): {total_tokens_all:,.0f}")
    else:
        print(f"⚠️  No request metadata found.")
else:
    print(f"ℹ️  Request/response metadata inspection skipped.")
    print(f"   Set INSPECT_METADATA=True to view API call details and token usage by model.")


ℹ️  Request/response metadata inspection skipped.
   Set INSPECT_METADATA=True to view API call details and token usage by model.


In [11]:
db_handler.close_connection()
print("✅ Database connection closed")

print(f"\n🎉 Questionnaire experiments complete!")
print(f"\n📊 Summary:")
print(f"   • Experiment groups created: {len(ALL_EXPERIMENT_GROUPS)}")
print(f"   • Total experiments executed: {len(ALL_REGISTERED_EXPERIMENTS)}")

if 'answers_df' in locals() and not answers_df.empty:
    print(f"   • Questionnaire answers collected: {len(answers_df)}")
    print(f"   • Unique models tested: {answers_df['model'].nunique()}")
    print(f"   • Unique populations tested: {answers_df['population'].nunique()}")

print(f"\n📋 Experiment Groups Summary:")
for group in ALL_EXPERIMENT_GROUPS:
    print(f"   • Group {group['group_id']}: {group['model']} / {group['population']} ({group['experiment_count']} experiments)")

print(f"\n🔬 Next Steps for Analysis:")
print(f"   • Calculate Big Five personality scores from answers")
print(f"   • Compare LLM responses across different models")
print(f"   • Analyze baseline vs borderline persona responses")
print(f"   • Study demographic bias patterns across models and populations")
print(f"   • Export data for statistical analysis")

print(f"\n💾 Data Export Example:")
print(f"   # Export all answers with model/population info")
print(f"   # answers_df.to_csv('all_questionnaire_answers.csv', index=False)")
print(f"   # ")
print(f"   # Export experiment status summary")
print(f"   # experiment_status.to_csv('all_experiment_status.csv', index=False)")
print(f"   # ")
print(f"   # Export per-group summaries")
print(f"   # summary_df.to_csv('experiment_summaries.csv', index=False)")


2025-10-09 18:23:22,697 - personas_backend.db.db_handler - INFO - Database connection closed successfully!


✅ Database connection closed

🎉 Questionnaire experiments complete!

📊 Summary:
   • Experiment groups created: 3
   • Total experiments executed: 6
   • Questionnaire answers collected: 144
   • Unique models tested: 2
   • Unique populations tested: 3

📋 Experiment Groups Summary:
   • Group 1: gpt35 / generated_gpt35_spain826 (2 experiments)
   • Group 2: gpt4o / borderline_maxN_gpt4o (2 experiments)
   • Group 3: gpt4o / borderline_maxP_gpt4o (2 experiments)

🔬 Next Steps for Analysis:
   • Calculate Big Five personality scores from answers
   • Compare LLM responses across different models
   • Analyze baseline vs borderline persona responses
   • Study demographic bias patterns across models and populations
   • Export data for statistical analysis

💾 Data Export Example:
   # Export all answers with model/population info
   # answers_df.to_csv('all_questionnaire_answers.csv', index=False)
   # 
   # Export experiment status summary
   # experiment_status.to_csv('all_experiment_s